# 2.4 External Tools

The following section demonstrates how to use the **signac** command line interface (CLI) in conjunction with other tools.

In [ ]:
%%bash
signac --help

To interact with a project on the command line, the current working directory needs to be within or below the project root directory.
Let's start by reseting the designated project root directory for this section of the tutorial.

In [ ]:
% pwd
% rm -rf projects/tutorial/cli
% mkdir -p projects/tutorial/cli
% cp idg projects/tutorial/cli

Next we switch the current working directory to the project root directory.

In [ ]:
% cd projects/tutorial/cli

Then we initialize the project.

In [ ]:
%%bash
signac init TutorialCLIProject

We can verify the project configuration.

In [ ]:
%%bash
signac project
signac project --workspace

We access the job handle by providing the state point on the command line in JSON format.

In [ ]:
%%bash
signac job '{"kT": 1.0, "p": 1.0, "N": 1000}'

The statepoints could also be read from STDIN, e.g., by reading it from a file.
Let's create a statepoints file with one statepoint:

In [ ]:
%%bash
echo '{"kT": 1.0, "p": 0.1, "N": 1000}' > statepoints.txt
echo '{"kT": 1.0, "p": 1.0, "N": 1000}' >> statepoints.txt
echo '{"kT": 1.0, "p": 10.0, "N": 1000}' >> statepoints.txt

cat statepoints.txt

We can pipe the content of this file into the **signac** CLI to get the corresponding *job id*.

In [ ]:
%%bash
head -n 1 statepoints.txt | signac job

Instead of the *job id*, we can directly obtain the path to the *job workspace*.

In [ ]:
%%bash
head -n 1 statepoints.txt | signac job --workspace

That's specifically useful in conjunction with external tools.
Let's pretend that we need to use a program called `idg` to calculate the ideal gas equation.

The `idg` program will calculate the volume of an ideal gas given the input parameters *p*, *kT* and *N*, just like in the previous sections.

In [ ]:
%%bash
./idg -p 1.0 -N 1000 --kT 1.0

We can store the result in a file based on the input arguments using the `-cw` argument, short for ``--create --workspace``, which returns the workspace path and creates it if necessary.

In [ ]:
%%bash
./idg -p 1.0 --kT 1.0 -N 1000 > $(signac job -cw '{"p": 1.0, "kT": 1.0, "N": 1000}')/V.txt

Obviously, we wouldn't write these commands all manually, but use a script instead.

In [ ]:
import signac
import json
project = signac.get_project()
for p in 0.1, 1.0, 10.0:
    job = project.open_job({'N': 1000, 'p': p, 'kT': 1.0})
    cmd = './idg -p {p} --kT {kT} -N {N}'.format(**job.statepoint())
    cmd += " > $(signac job -cw '{}')/V.txt".format(json.dumps(job.statepoint()))
    print(cmd)

We can then execute this script...

In [ ]:
%%bash
./idg -p 0.1 --kT 1.0 -N 1000 > $(signac job -cw '{"kT": 1.0, "p": 0.1, "N": 1000}')/V.txt
./idg -p 1.0 --kT 1.0 -N 1000 > $(signac job -cw '{"kT": 1.0, "p": 1.0, "N": 1000}')/V.txt
./idg -p 10.0 --kT 1.0 -N 1000 > $(signac job -cw '{"kT": 1.0, "p": 10.0, "N": 1000}')/V.txt

... and examine the results.

In [ ]:
% cat `signac project -w`/*/V.txt